# קריפטוגרפיה

מחברת אינטראקטיבית להבנת עקרונות קריפטוגרפיה: ראשוניות, Diffie-Hellman, ו-RSA.

## 1. בדיקת ראשוניות

In [ ]:
def is_prime_naive(n):
    """בדיקה נאיבית - O(n)"""
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

def is_prime(n):
    """בדיקה יעילה - O(√n)"""
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    i = 3
    while i * i <= n:
        if n % i == 0:
            return False
        i += 2
    return i

# דוגמאות
test_numbers = [1, 2, 3, 4, 17, 18, 97, 100, 561]
for n in test_numbers:
    print(f"{n}: {'ראשוני' if is_prime(n) else 'לא ראשוני'}")

## 2. נפה של ארטוסתנס

In [ ]:
def sieve(n):
    """נפה של ארטוסתנס - כל הראשוניים עד n"""
    is_prime_arr = [True] * (n + 1)
    is_prime_arr[0] = is_prime_arr[1] = False
    
    p = 2
    while p * p <= n:
        if is_prime_arr[p]:
            # סמן את כפולות p כלא ראשוניים
            for i in range(p * p, n + 1, p):
                is_prime_arr[i] = False
        p += 1
    
    return [i for i in range(n + 1) if is_prime_arr[i]]

primes = sieve(100)
print(f"ראשוניים עד 100: {primes}")
print(f"כמות: {len(primes)}")

## 3. חשבון מודולרי

In [ ]:
def mod_power(base, exp, mod):
    """חישוב base^exp mod mod - יעיל!"""
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:  # אם exp אי-זוגי
            result = (result * base) % mod
        exp = exp // 2
        base = (base * base) % mod
    return result

# דוגמה: 3^100 mod 7
result = mod_power(3, 100, 7)
print(f"3^100 mod 7 = {result}")

# השוואה לחישוב ישיר (רק למספרים קטנים)
print(f"אימות: {pow(3, 100, 7)}")

## 4. אלגוריתם אוקלידס - GCD

In [ ]:
def gcd(a, b):
    """מחלק משותף מקסימלי - אלגוריתם אוקלידס"""
    while b:
        a, b = b, a % b
    return a

def extended_gcd(a, b):
    """GCD מורחב - מחזיר gcd, x, y כך ש ax + by = gcd"""
    if b == 0:
        return a, 1, 0
    gcd_val, x, y = extended_gcd(b, a % b)
    return gcd_val, y, x - (a // b) * y

# דוגמאות
print(f"gcd(48, 18) = {gcd(48, 18)}")
print(f"gcd(17, 13) = {gcd(17, 13)}")

g, x, y = extended_gcd(17, 13)
print(f"\nextended_gcd(17, 13) = {g}")
print(f"17 * {x} + 13 * {y} = {17*x + 13*y}")

## 5. Diffie-Hellman Key Exchange

In [ ]:
import random

def diffie_hellman_demo(p, g):
    """הדגמת החלפת מפתחות Diffie-Hellman"""
    print(f"פרמטרים ציבוריים: p={p}, g={g}")
    print()
    
    # Alice בוחרת מפתח פרטי
    a = random.randint(2, p - 2)
    A = pow(g, a, p)  # המפתח הציבורי של Alice
    print(f"Alice: מפתח פרטי a={a}, מפתח ציבורי A=g^a mod p = {A}")
    
    # Bob בוחר מפתח פרטי
    b = random.randint(2, p - 2)
    B = pow(g, b, p)  # המפתח הציבורי של Bob
    print(f"Bob:   מפתח פרטי b={b}, מפתח ציבורי B=g^b mod p = {B}")
    print()
    
    # חישוב המפתח המשותף
    K_alice = pow(B, a, p)  # Alice מחשבת B^a mod p
    K_bob = pow(A, b, p)    # Bob מחשב A^b mod p
    
    print(f"Alice מחשבת: K = B^a mod p = {K_alice}")
    print(f"Bob מחשב:   K = A^b mod p = {K_bob}")
    print(f"\nהמפתח המשותף הסודי: {K_alice}")
    
    return K_alice == K_bob

# הדגמה עם מספרים קטנים
diffie_hellman_demo(p=23, g=5)

## 6. RSA - הקמת מפתחות

In [ ]:
def mod_inverse(e, phi):
    """מציאת d כך ש e*d ≡ 1 (mod phi)"""
    g, x, _ = extended_gcd(e, phi)
    if g != 1:
        return None  # אין הופכי
    return x % phi

def generate_rsa_keys(p, q):
    """יצירת מפתחות RSA"""
    # שלב 1: חישוב n ו-phi
    n = p * q
    phi = (p - 1) * (q - 1)
    
    print(f"p = {p}, q = {q}")
    print(f"n = p * q = {n}")
    print(f"φ(n) = (p-1)(q-1) = {phi}")
    
    # שלב 2: בחירת e (בד"כ 65537)
    e = 65537
    if gcd(e, phi) != 1:
        # מצא e אחר
        e = 3
        while gcd(e, phi) != 1:
            e += 2
    
    # שלב 3: חישוב d
    d = mod_inverse(e, phi)
    
    print(f"\nמפתח ציבורי: (e={e}, n={n})")
    print(f"מפתח פרטי:  (d={d}, n={n})")
    print(f"\nאימות: e * d mod φ(n) = {(e * d) % phi}")
    
    return (e, n), (d, n)

# יצירת מפתחות
public_key, private_key = generate_rsa_keys(p=61, q=53)

## 7. RSA - הצפנה ופענוח

In [ ]:
def rsa_encrypt(message, public_key):
    """הצפנת מספר בודד"""
    e, n = public_key
    return pow(message, e, n)

def rsa_decrypt(ciphertext, private_key):
    """פענוח מספר בודד"""
    d, n = private_key
    return pow(ciphertext, d, n)

# הדגמה
message = 42
print(f"הודעה מקורית: {message}")

encrypted = rsa_encrypt(message, public_key)
print(f"הודעה מוצפנת: {encrypted}")

decrypted = rsa_decrypt(encrypted, private_key)
print(f"הודעה מפוענחת: {decrypted}")
print(f"\nהצלחה: {message == decrypted}")

## 8. הצפנת טקסט מלא

In [ ]:
def rsa_encrypt_string(text, public_key):
    """הצפנת מחרוזת (תו-תו)"""
    return [rsa_encrypt(ord(ch), public_key) for ch in text]

def rsa_decrypt_string(ciphertext, private_key):
    """פענוח מחרוזת"""
    return ''.join(chr(rsa_decrypt(c, private_key)) for c in ciphertext)

# הדגמה
message = "HELLO"
print(f"הודעה: '{message}'")

encrypted = rsa_encrypt_string(message, public_key)
print(f"מוצפן: {encrypted}")

decrypted = rsa_decrypt_string(encrypted, private_key)
print(f"מפוענח: '{decrypted}'")

## 9. חתימה דיגיטלית

In [ ]:
def simple_hash(message):
    """פונקציית גיבוב פשוטה (לא מאובטחת!)"""
    h = 0
    for ch in message:
        h = (h * 31 + ord(ch)) % 1000000
    return h

def sign(message, private_key):
    """יצירת חתימה"""
    h = simple_hash(message)
    d, n = private_key
    return pow(h, d, n)  # הצפנה עם המפתח הפרטי

def verify(message, signature, public_key):
    """אימות חתימה"""
    h = simple_hash(message)
    e, n = public_key
    decrypted_hash = pow(signature, e, n)  # פענוח עם המפתח הציבורי
    return h == decrypted_hash

# הדגמה
message = "Transfer 100 to Bob"
signature = sign(message, private_key)

print(f"הודעה: '{message}'")
print(f"חתימה: {signature}")
print(f"אימות: {verify(message, signature, public_key)}")

# ניסיון זיוף
fake_message = "Transfer 1000 to Bob"
print(f"\nהודעה מזויפת: '{fake_message}'")
print(f"אימות עם אותה חתימה: {verify(fake_message, signature, public_key)}")

## 10. סיכום נוסחאות

| נושא | נוסחה | הערות |
|------|-------|-------|
| בדיקת ראשוניות | בדוק חלוקה עד √n | O(√n) |
| GCD | gcd(a, b) = gcd(b, a mod b) | O(log min(a,b)) |
| Diffie-Hellman | K = g^(ab) mod p | a,b סודיים |
| RSA יצירה | n=pq, φ=(p-1)(q-1), ed≡1(mod φ) | p,q ראשוניים גדולים |
| RSA הצפנה | c = m^e mod n | e ציבורי |
| RSA פענוח | m = c^d mod n | d פרטי |
| חתימה | sig = hash(m)^d mod n | פענוח עם e לאימות |